In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import optuna

In [2]:
df = pd.read_csv("../input/30days-folds/train_folds.csv")
df_test = pd.read_csv("../input/30-days-of-ml/test.csv")
sample_submission = pd.read_csv("../input/30-days-of-ml/sample_submission.csv")

useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if col.startswith("cat")]
df_test = df_test[useful_features]

for col in object_cols:
    temp_df = []
    temp_test_feat = None
    for fold in range(5):
        xtrain =  df[df.kfold != fold].reset_index(drop=True)
        xvalid = df[df.kfold == fold].reset_index(drop=True)
        feat = xtrain.groupby(col)["target"].agg("mean")
        feat = feat.to_dict()
        xvalid.loc[:, f"tar_enc_{col}"] = xvalid[col].map(feat)
        temp_df.append(xvalid)
        if temp_test_feat is None:
            temp_test_feat = df_test[col].map(feat)
        else:
            temp_test_feat += df_test[col].map(feat)
    
    temp_test_feat /= 5
    df_test.loc[:, f"tar_enc_{col}"] = temp_test_feat
    df = pd.concat(temp_df)
    

useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if col.startswith("cat")]
df_test = df_test[useful_features]

In [6]:
def run(trial):
    fold = 0
    learning_rate = trial.suggest_float("learning_rate", 1e-2, 0.25, log=True)
    reg_lambda = trial.suggest_loguniform("reg_lambda", 1e-8, 100.0)
    reg_alpha = trial.suggest_loguniform("reg_alpha", 1e-8, 100.0)
    subsample = trial.suggest_float("subsample", 0.1, 1.0)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.1, 1.0)
    max_depth = trial.suggest_int("max_depth", 1, 7)

    xtrain = df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)

    ytrain = xtrain.target
    yvalid = xvalid.target

    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]

    ordinal_encoder = preprocessing.OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])

    model = XGBRegressor(
        random_state=1,
        tree_method="gpu_hist",
        gpu_id=1,
        predictor="gpu_predictor",
        n_estimators=7000,
        learning_rate=learning_rate,
        reg_lambda=reg_lambda,
        reg_alpha=reg_alpha,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        max_depth=max_depth,
    )
    model.fit(xtrain, ytrain, early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=1000)
    preds_valid = model.predict(xvalid)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    return rmse

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(run, n_trials=20)

[I 2021-08-31 20:24:18,613] A new study created in memory with name: no-name-02ef21b6-e409-4f47-a5f7-c12ae47c4e06


[20:24:21] WARNING: ../src/learner.cc:231: Only 1 GPUs are visible, setting `gpu_id` to 0
[0]	validation_0-rmse:6.42606
[819]	validation_0-rmse:0.72254


[I 2021-08-31 20:24:24,242] Trial 0 finished with value: 0.7213835125554788 and parameters: {'learning_rate': 0.17596830002500205, 'reg_lambda': 13.731907662472675, 'reg_alpha': 0.06238251960575841, 'subsample': 0.2334730434956424, 'colsample_bytree': 0.10523036968287411, 'max_depth': 4}. Best is trial 0 with value: 0.7213835125554788.


[20:24:26] WARNING: ../src/learner.cc:231: Only 1 GPUs are visible, setting `gpu_id` to 0
[0]	validation_0-rmse:6.95002
[1000]	validation_0-rmse:0.72141
[2000]	validation_0-rmse:0.71891
[3000]	validation_0-rmse:0.71819
[3487]	validation_0-rmse:0.71815


[I 2021-08-31 20:24:36,299] Trial 1 finished with value: 0.7180840032669827 and parameters: {'learning_rate': 0.10774417504372416, 'reg_lambda': 0.00032788048988744025, 'reg_alpha': 22.336766671382403, 'subsample': 0.8013077426974943, 'colsample_bytree': 0.41056829411803075, 'max_depth': 2}. Best is trial 1 with value: 0.7180840032669827.


...

[20:27:08] WARNING: ../src/learner.cc:231: Only 1 GPUs are visible, setting `gpu_id` to 0
[0]	validation_0-rmse:7.28892
[1000]	validation_0-rmse:0.71914
[1490]	validation_0-rmse:0.71936


[I 2021-08-31 20:27:15,586] Trial 14 finished with value: 0.71907634248476 and parameters: {'learning_rate': 0.06371546041128873, 'reg_lambda': 0.0354309519373568, 'reg_alpha': 0.0029697135767132125, 'subsample': 0.9967529285066266, 'colsample_bytree': 0.2547215528867274, 'max_depth': 5}. Best is trial 1 with value: 0.7180840032669827.


[20:27:18] WARNING: ../src/learner.cc:231: Only 1 GPUs are visible, setting `gpu_id` to 0
[0]	validation_0-rmse:5.98266
[1000]	validation_0-rmse:0.72679
[2000]	validation_0-rmse:0.72446
[3000]	validation_0-rmse:0.72326
[4000]	validation_0-rmse:0.72236
[5000]	validation_0-rmse:0.72173
[6000]	validation_0-rmse:0.72124
[6999]	validation_0-rmse:0.72083


[I 2021-08-31 20:27:33,767] Trial 15 finished with value: 0.72080237739145 and parameters: {'learning_rate': 0.2335956345433736, 'reg_lambda': 5.589949505133686e-07, 'reg_alpha': 96.72557180440127, 'subsample': 0.7148016688109191, 'colsample_bytree': 0.4689355381038265, 'max_depth': 1}. Best is trial 1 with value: 0.7180840032669827.


[20:27:36] WARNING: ../src/learner.cc:231: Only 1 GPUs are visible, setting `gpu_id` to 0
[0]	validation_0-rmse:7.39353
[1000]	validation_0-rmse:0.72177
[2000]	validation_0-rmse:0.71890
[3000]	validation_0-rmse:0.71803
[4000]	validation_0-rmse:0.71778
[4407]	validation_0-rmse:0.71778


In [5]:
study.best_params

{'learning_rate': 0.10060669515592832,
 'reg_lambda': 3.6268609366263594e-08,
 'reg_alpha': 0.5837933768291012,
 'subsample': 0.9420844926252625,
 'colsample_bytree': 0.6067936212382514,
 'max_depth': 2}